In [1]:
# Delete this cell to re-enable tracebacks
import sys
import traceback
ipython = get_ipython()

def hide_traceback(exc_tuple=None, filename=None, tb_offset=None,
                   exception_only=False, running_compiled_code=False):
    etype, value, tb = sys.exc_info()    
    value.__cause__ = None  # suppress chained exceptions
    return ipython._showtraceback(etype, value, ipython.InteractiveTB.get_exception_only(etype, value))

ipython.showtraceback = hide_traceback

In [2]:
# JSON output syntax highlighting
from __future__ import print_function
from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import HtmlFormatter
from IPython.display import HTML

original_print = print

def json_print(inpt):
    string = str(inpt)
    if string[0] == '{':
        formatter = HtmlFormatter()
        return HTML('<style type="text/css">{}</style>{}'.format(
                    formatter.get_style_defs('.highlight'),
                    highlight(string, JsonLexer(), formatter)))
    else:
        original_print(inpt)

print = json_print

## Creating STIX Content

### Creating STIX Domain Objects

To create a STIX object, provide keyword arguments to the type's constructor:

In [15]:
from stix2 import Indicator

indicator = Indicator(name="File hash for malware variant",
                      pattern="[file:hashes.md5 = 'd41d8cd98f00b204e9800998ecf8427e']",
                      pattern_type="stix")
print(indicator)

Certain required attributes of all objects will be set automatically if not provided as keyword arguments:

-  If not provided, ``type`` will be set automatically to the correct type. You can also provide the type explicitly, but this is not necessary:

In [16]:
indicator2 = Indicator(type='indicator',
                       pattern_type="stix",
                       pattern="[file:hashes.md5 = 'd41d8cd98f00b204e9800998ecf8427e']")

Passing a value for ``type`` that does not match the class being constructed will cause an error:

In [17]:
indicator3 = Indicator(type='xxx',
                       pattern_type="stix",
                       pattern="[file:hashes.md5 = 'd41d8cd98f00b204e9800998ecf8427e']")

InvalidValueError: Invalid value for Indicator 'type': must equal 'indicator'.

-  If not provided, ``id`` will be generated randomly. If you provide an
   ``id`` argument, it must begin with the correct prefix:

In [18]:
indicator4 = Indicator(id="campaign--63ce9068-b5ab-47fa-a2cf-a602ea01f21a",
                       pattern_type="stix",
                       pattern="[file:hashes.md5 = 'd41d8cd98f00b204e9800998ecf8427e']")

InvalidValueError: Invalid value for Indicator 'id': must start with 'indicator--'.

For indicators, ``pattern`` and ``pattern_type`` are required and cannot be set automatically. Trying to create an indicator that is missing one of these properties will result in an error:

In [8]:
indicator = Indicator()

MissingPropertiesError: No values for required properties for Indicator: (pattern, pattern_type).

However, the required ``valid_from`` attribute on Indicators will be set to the current time if not provided as a keyword argument.

Once created, the object acts like a frozen dictionary. Properties can be accessed using the standard Python dictionary syntax:

In [9]:
indicator['name']

'File hash for malware variant'

Or access properties using the standard Python attribute syntax:

In [10]:
indicator.name

'File hash for malware variant'

<div class="alert alert-warning">

**Warning**

Note that there are several attributes on these objects used for method names.  Accessing those will return a bound method, not the attribute value.

</div>


Attempting to modify any attributes will raise an error:

In [11]:
indicator['name'] = "This is a revised name"

TypeError: 'Indicator' object does not support item assignment

In [12]:
indicator.name = "This is a revised name"

ImmutableError: Cannot modify 'name' property in 'Indicator' after creation.

To update the properties of an object, see the [Versioning](versioning.ipynb) section.

Creating a Malware object follows the same pattern:

In [14]:
from stix2 import Malware

malware = Malware(name="Poison Ivy",
                  is_family=False)
print(malware)

As with indicators, the ``type``, ``id``, ``created``, and ``modified`` properties will be set automatically if not provided. For Malware objects, the ``is_family`` property must be provided.

You can see the full list of SDO classes [here](../api/v21/stix2.v21.sdo.rst).

### Creating Relationships

STIX 2 Relationships are separate objects, not properties of the object on either side of the relationship. They are constructed similarly to other STIX objects. The ``type``, ``id``, ``created``, and ``modified`` properties are added automatically if not provided. Callers must provide the ``relationship_type``, ``source_ref``, and ``target_ref`` properties.

In [19]:
from stix2 import Relationship

relationship = Relationship(relationship_type='indicates',
                            source_ref=indicator.id,
                            target_ref=malware.id)
print(relationship)

The ``source_ref`` and ``target_ref`` properties can be either the ID's of other STIX objects, or the STIX objects themselves. For readability, Relationship objects can also be constructed with the ``source_ref``, ``relationship_type``, and ``target_ref`` as positional (non-keyword) arguments:

In [20]:
relationship2 = Relationship(indicator, 'indicates', malware)
print(relationship2)

### Creating Bundles

STIX Bundles can be created by passing objects as arguments to the Bundle constructor. All required properties (``type``, ``id``, and ``spec_version``) will be set automatically if not provided, or can be provided as keyword arguments:

In [21]:
from stix2 import Bundle

bundle = Bundle(indicator, malware, relationship)
print(bundle)

### Creating Cyber Observable References
Cyber Observable Objects have properties that can reference other Cyber Observable Objects. In order to create those references, either supply the ID string of the object being referenced, or pass in the object itself.

For example, the IPv4Address object has a ``resolves_to_refs`` property which must hold a list of references to MACAddress objects. We could specify the id string:

In [22]:
from stix2 import IPv4Address

ip4 = IPv4Address(
    value="177.60.40.7",
    resolves_to_refs=["mac-addr--43f380fd-37c6-476d-8643-60849bf9240e"]
)

print(ip4)

Or we could create the MACAddress object(s) beforehand and then pass them in:

In [23]:
from stix2 import MACAddress

mac_addr_a = MACAddress(value="a1:b2:c3:d4:e5:f6")
mac_addr_b = MACAddress(value="a7:b8:c9:d0:e1:f2")

ip4_valid_refs = IPv4Address(
    value="177.60.40.7",
    resolves_to_refs=[mac_addr_a.id, mac_addr_b.id]
)

print(ip4_valid_refs)